In [ ]:
%matplotlib inline

import ROOT, sys, os
from ROOT import std

from larcv import larcv
from larlite import larlite as ll
from larlite import larutil as lu

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.path as path
import matplotlib.patches as patches

from ROOT import geo2d,cv
from ROOT.cv import Point_ as Vector
DTYPE='float'

import root_numpy as rn
import pandas as pd

geoh=lu.GeometryHelper.GetME()
geo=lu.Geometry.GetME()
larp=lu.LArProperties.GetME()
pygeo = geo2d.PyDraw()

In [ ]:
reco_df = pd.DataFrame(rn.root2rec("../reco_ana.root",treename='LArbysImageTree'))
mc_df   = pd.DataFrame(rn.root2rec("../mc_ana.root",treename='mctree'))

In [ ]:
iom=larcv.IOManager(larcv.IOManager.kBOTH)        
iom.reset()
iom.set_verbosity(0)                                                                          
iom.add_in_file("/Users/vgenty/Desktop/out_0000_0099.root")
iom.set_out_file("/tmp/trash.root")
iom.initialize()

larbysimg=larcv.LArbysImage()
cfg=larcv.CreatePSetFromFile("empty.fcl","LArbysImage")
larbysimg.configure(cfg)
larbysimg.initialize()

In [ ]:
for entry in reco_idx.values:
    iom.read_entry(entry)
    larbysimg.process(iom)
    mgr=larbysimg.Manager()
    
    img_v = []

    fig,ax=plt.subplots(figsize=(20,20))
    maxx=0;minx=9999
    maxy=0;miny=9999
    plane=0

    caca_img=np.zeros((485, 485))

    for mat in mgr.InputImages():

        img_v.append(pygeo.image(mat))
        shape_img=np.where(img_v[plane]>1.5,1.0,0.0).astype(np.uint8)

        plane+=1

        nz_pixels=np.where(shape_img>0.0)
        
        maxx=np.maximum(maxx,np.max(nz_pixels[1])+100)
        minx=np.minimum(minx,np.min(nz_pixels[1])-100)

        maxy=np.maximum(maxy,np.max(nz_pixels[0])+100)
        miny=np.minimum(miny,np.min(nz_pixels[0])-100)

        caca_img+=shape_img

    caca_img=np.where(caca_img>0,1.0,0.0)

    plt.imshow(caca_img,cmap='Greys',interpolation='none')

    for iy in xrange(reco_df.iloc[entry].n_circle_vtx):
        x_v=reco_df.iloc[entry].circle_vtx_x_vv[iy]
        y_v=reco_df.iloc[entry].circle_vtx_y_vv[iy]

        for x,y in zip(x_v,y_v):
            plt.plot(x,y,'*',color='orange',markersize=20)
        
        w_v=mc_df.iloc[entry].vtx2d_w
        t_v=mc_df.iloc[entry].vtx2d_t
        
        for w,t in zip(w_v,t_v):
            plt.plot(t,w,'*',color='yellow',markersize=20)

    ax.set_title("Entry {}".format(entry),fontweight='bold',fontsize=40)
    #plt.savefig("reco_%04d.png"%entry)
    break
    plt.show()

In [ ]:
reco_df_copy = reco_df.copy()

cuts_v=[]
cuts_v.append('n_vtx3d>0')

for cut in cuts_v:
    reco_df_copy = reco_df_copy.query(cut)

In [ ]:
reco_df_copy = reco_df_copy[reco_df_copy['vtx3d_num_planes_v'].apply(lambda x : np.where(x==3)[0].size)>0]

In [ ]:
def pixels(x):
    y=np.hstack(x)
    return np.where(y>0.9)[0].size

reco_idx=reco_df_copy[reco_df_copy['num_pixel_frac_vv'].apply(pixels) > 0].index

In [ ]:
np3_v=reco_df.ix[reco_idx].vtx3d_num_planes_v.apply(lambda x : np.where(x==3)[0][0]).values

In [ ]:
dw_v=[]
dt_v=[]
for ix,idx in enumerate(reco_idx):
    mc_t   = mc_df.ix[idx].vtx2d_w
    mc_w   = mc_df.ix[idx].vtx2d_t
    reco_w = reco_df.ix[idx].circle_vtx_x_vv[np3_v[ix]]
    reco_t = reco_df.ix[idx].circle_vtx_y_vv[np3_v[ix]]
    dw = mc_w - reco_w
    dt = mc_t - reco_t
    dw_v.append(dw)
    dt_v.append(dt)
dw_v = np.array(dw_v)
dt_v = np.array(dt_v)

In [ ]:
matplotlib.rcParams['font.size']=20
matplotlib.rcParams['font.family']='serif'
fig,ax=plt.subplots(figsize=(10,6))
for i in xrange(3):
    ax.hist(dw_v[:,i],label='Plane {}'.format(i),bins=np.arange(-80,80,2),alpha=0.5)
ax.set_xlabel("mc_w - reco_w [pixel]",fontweight='bold')
ax.legend()
plt.grid()
plt.show()

fig,ax=plt.subplots(figsize=(10,6))
for i in xrange(3):
    ax.hist(dt_v[:,i],label='Plane {}'.format(i),bins=np.arange(-80,80,2),alpha=0.5)
ax.set_xlabel("mc_t - reco_t [pixel]",fontweight='bold')
ax.legend()
plt.grid()
plt.show()


for i in xrange(3):
    fig,ax=plt.subplots(figsize=(10,6))
    ax.hist(np.sqrt(np.power(dw_v[:,i],2)+np.power(dt_v[:,i],2)),
            label='Plane {}'.format(i),bins=np.arange(0,100,1),alpha=0.5,lw=2)
    ax.set_xlabel("dist 2D mc->reco [pixel]",fontweight='bold')
    ax.set_ylim(0,15)
    ax.legend()
    plt.grid()
    plt.show()
